# Load Data

In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
%pip install polars==1.25.2 >> _

Note: you may need to restart the kernel to use updated packages.


����� �� ����� ������� ����� � 䠩��, ⠪ ��� ��� 䠩� ����� ��㣨� ����ᮬ.


In [3]:
%pip install implicit >> _

Note: you may need to restart the kernel to use updated packages.


����� �� ����� ������� ����� � 䠩��, ⠪ ��� ��� 䠩� ����� ��㣨� ����ᮬ.


In [4]:
%pip install numpy >> _
%pip install scipy >> _


����� �� ����� ������� ����� � 䠩��, ⠪ ��� ��� 䠩� ����� ��㣨� ����ᮬ.


Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.


����� �� ����� ������� ����� � 䠩��, ⠪ ��� ��� 䠩� ����� ��㣨� ����ᮬ.


In [5]:
%mkdir data

�������� ��� 䠩� data 㦥 �������.


In [6]:
# # takes 5 minutes
# !wget https://storage.yandexcloud.net/ds-ods/files/data/docs/competitions/Avitotechcomp2025/data_competition_1/clickstream.pq -O data/clickstream.pq >> _
# !wget https://storage.yandexcloud.net/ds-ods/files/data/docs/competitions/Avitotechcomp2025/data_competition_1/test_users.pq -O data/test_users.pq >> _
# !wget https://storage.yandexcloud.net/ds-ods/files/data/docs/competitions/Avitotechcomp2025/data_competition_1/cat_features.pq -O data/cat_features.pq >> _
# !wget https://storage.yandexcloud.net/ds-ods/files/data/docs/competitions/Avitotechcomp2025/data_competition_1/text_features.pq -O data/text_features.pq >> _
# !wget https://storage.yandexcloud.net/ds-ods/files/data/docs/competitions/Avitotechcomp2025/data_competition_1/events.pq -O data/events.pq >> _


In [7]:
from datetime import timedelta
import polars as pl
import implicit

ModuleNotFoundError: No module named 'implicit'

In [ ]:
DATA_DIR = '/kaggle/input/avito-cup-2025-recsys/'

df_test_users = pl.read_parquet(f'{DATA_DIR}/test_users.pq')
df_clickstream = pl.read_parquet(f'{DATA_DIR}/clickstream.pq')

df_cat_features = pl.read_parquet(f'{DATA_DIR}/cat_features.pq')
df_text_features = pl.read_parquet(f'{DATA_DIR}/text_features.pq')
df_event = pl.read_parquet(f'{DATA_DIR}/events.pq')

# PREPARE TRAIN EVAL

In [ ]:
EVAL_DAYS_TRESHOLD = 14

In [ ]:
treshhold = df_clickstream['event_date'].max() - timedelta(days=EVAL_DAYS_TRESHOLD)

In [ ]:
df_train = df_clickstream.filter(df_clickstream['event_date']<= treshhold)
df_eval = df_clickstream.filter(df_clickstream['event_date']> treshhold)[['cookie', 'node', 'event']]

In [ ]:
df_eval = df_eval.join(df_train, on=['cookie', 'node'], how='anti')


In [ ]:
df_eval = df_eval.filter(
    pl.col('event').is_in(
        df_event.filter(pl.col('is_contact')==1)['event'].unique()
    )
)

In [ ]:
df_eval = df_eval.filter(
        pl.col('cookie').is_in(df_train['cookie'].unique())
    ).filter(
        pl.col('node').is_in(df_train['node'].unique())
    )

# TRAIN MODEL

## ALS

In [ ]:
def get_als_pred(users, nodes, user_to_pred, 
                 factors=60, 
                 iterations=10, 
                 regularization=0.1, 
                 alpha=1.0,
                 use_cg=True,
                 random_state=None):
    """
    Получение предсказаний ALS модели
    
    Параметры:
    ----------
    users : pd.Series / pl.Series
        Серия с идентификаторами пользователей
    nodes : pd.Series / pl.Series
        Серия с идентификаторами товаров/узлов
    user_to_pred : list
        Список пользователей для предсказания
    factors : int, optional
        Количество латентных факторов (по умолчанию 60)
    iterations : int, optional
        Количество итераций обучения (по умолчанию 10)
    regularization : float, optional
        Параметр регуляризации (по умолчанию 0.1)
    alpha : float, optional
        Параметр доверия (по умолчанию 1.0)
    use_cg : bool, optional
        Использовать ли метод сопряженных градиентов (по умолчанию True)
    random_state : int, optional
        Seed для воспроизводимости (по умолчанию None)
    
    Возвращает:
    -----------
    pl.DataFrame
        Датафрейм с предсказаниями (columns: cookie, node, scores)
    """
    user_ids = users.unique().to_list()
    item_ids = nodes.unique().to_list()
        
    user_id_to_index = {user_id: idx for idx, user_id in enumerate(user_ids)}
    item_id_to_index = {item_id: idx for idx, item_id in enumerate(item_ids)}
    index_to_item_id = {v:k for k,v in item_id_to_index.items()}
    
    rows = users.replace_strict(user_id_to_index).to_list()
    cols = nodes.replace_strict(item_id_to_index).to_list()
    
    values = [1] * len(users)
    
    sparse_matrix = csr_matrix((values, (rows, cols)), shape=(len(user_ids), len(item_ids)))
    
    # Инициализация модели с переданными параметрами
    model = implicit.als.AlternatingLeastSquares(
        factors=factors,
        iterations=iterations,
        regularization=regularization,
        alpha=alpha,
        use_cg=use_cg,
        random_state=random_state
    )
    
    # Обучение модели с учетом параметра alpha
    model.fit((sparse_matrix * alpha).astype('float64'))
    
    user4pred = np.array([user_id_to_index[i] for i in user_to_pred])
    
    recommendations, scores = model.recommend(
        user4pred, 
        sparse_matrix[user4pred], 
        N=40, 
        filter_already_liked_items=True
    )
    
    df_pred = pl.DataFrame(
        {
            'node': [
                [index_to_item_id[i] for i in i] for i in recommendations.tolist()
            ], 
            'cookie': list(user_to_pred),
            'scores': scores.tolist()
        }
    )
    df_pred = df_pred.explode(['node', 'scores'])
    return df_pred

In [ ]:
from scipy.sparse import csr_matrix
import numpy as np
import implicit


users = df_train["cookie"]
nodes = df_train["node"]
eval_users = df_eval['cookie'].unique().to_list()

df_pred = get_als_pred(users, nodes,eval_users )

## popular

In [ ]:
def get_popular(df):
    popukar_node = df.group_by('node').agg(pl.col('cookie').count()).sort('cookie').tail(40)['node'].to_list()
    df_pred_pop = pl.DataFrame({'node': [popukar_node for i in range(len(eval_users))], 'cookie': eval_users})
    df_pred_pop = df_pred_pop.explode('node')
    return df_pred_pop

train_pop = get_popular(df_train)


# CALC EVAL METRICS

In [ ]:
def recall_at(df_true, df_pred, k=40):
    return  df_true[['node', 'cookie']].join(
        df_pred.group_by('cookie').head(k).with_columns(value=1)[['node', 'cookie', 'value']], 
        how='left',
        on = ['cookie', 'node']
    ).select(
        [pl.col('value').fill_null(0), 'cookie']
    ).group_by(
        'cookie'
    ).agg(
        [
            pl.col('value').sum()/pl.col(
                'value'
            ).count()
        ]
    )['value'].mean()


In [ ]:
recall_at(df_eval, df_pred, k=40)

In [ ]:
recall_at(df_eval, train_pop, k=40)

# SUMBIT

In [ ]:
users = df_clickstream["cookie"]
nodes = df_clickstream["node"]
test_users = df_test_users['cookie'].unique().to_list()

df_pred = get_als_pred(users, nodes, test_users )


In [ ]:
df_pred.write_csv('prediction.csv')

# ML FLOW SETUP

In [ ]:
%pip install mlflow >> _

In [ ]:
import mlflow

In [ ]:
mlflow.set_tracking_uri("http://51.250.35.156:5000/")

In [ ]:
NAME="homework-ralolaev.ext"

In [ ]:
# mlflow.create_experiment(
#     name=NAME,
#     artifact_location="mlflow-artifacts:/"
# )

In [ ]:
mlflow.set_experiment(experiment_name=NAME)

In [ ]:
from scipy.sparse import csr_matrix
import numpy as np
import implicit
import time

users = df_train["cookie"]
nodes = df_train["node"]
eval_users = df_eval['cookie'].unique().to_list()

In [ ]:
als_params = {
    'factors': 100,
    'iterations': 10, 
    'regularization': 0.005,
    'alpha': 1,
    'use_cg': False,
    'random_state': 42
}

with mlflow.start_run(run_name="ALS_best_Run"):
    try:
        mlflow.log_params(als_params)
        
        df_pred = get_als_pred(users, nodes, eval_users, **als_params)
        
        r_als = recall_at(df_eval, df_pred, k=40)
        mlflow.log_metric("Recall_40", r_als)
        
        print(f"ALS Recall@40: {r_als:.4f}")
        
    except Exception as e:
        mlflow.log_param("error", str(e))
        raise

# Перебор параметров ALS

In [ ]:
# import mlflow
# import itertools
# import time
# import polars as pl

# param_grid_als = {
#     'factors': [50, 100, 150, 200],
#     'iterations': [10, 15, 20, 25],
#     'regularization': [0.005, 0.01, 0.05, 0.1],
#     'alpha': [1, 5, 10, 20, 40],
#     'use_cg': [False],
#     'random_state': [42]
# }

# param_names_als = list(param_grid_als.keys())
# param_value_lists_als = list(param_grid_als.values())

# all_param_combinations_als = list(itertools.product(*param_value_lists_als))

# print(f"Запланировано запусков для ALS: {len(all_param_combinations_als)}")

# for i, param_values_tuple_als in enumerate(all_param_combinations_als):
#     current_als_params = dict(zip(param_names_als, param_values_tuple_als))

#     run_name_als = f"ALS_f{current_als_params['factors']}" \
#                    f"_i{current_als_params['iterations']}" \
#                    f"_reg{current_als_params['regularization']}" \
#                    f"_alpha{current_als_params['alpha']}" \
#                    f"_cg{'T' if current_als_params['use_cg'] else 'F'}"

#     print(f"\n--- Запуск ALS {i+1}/{len(all_param_combinations_als)}: {run_name_als} ---")
#     print(f"Параметры: {current_als_params}")

#     with mlflow.start_run(run_name=run_name_als):
#         mlflow.log_params(current_als_params)
#         mlflow.log_param("model_type", "ALS")

#         try:
#             start_time_train = time.time()
            
#             df_pred = get_als_pred(
#                 users, nodes, eval_users, **current_als_params
#             )
            
#             training_time = time.time() - start_time_train
#             mlflow.log_metric("training_time_sec", training_time)
#             print(f"Время обучения: {training_time:.2f} сек.")

#             if df_pred is not None and df_pred.height > 0:
#                 r_als = recall_at(df_eval, df_pred, k=40)
#                 mlflow.log_metric("Recall_40", r_als)
#                 print(f"ALS Recall@40: {r_als:.4f}")
                                
#             elif df_pred is None:
#                  print("ALS предсказания: df_pred is None.")
#                  mlflow.log_metric("Recall_40", 0.0)
#             else: # df_pred.height == 0
#                 print("ALS предсказания: df_pred пустой (нет строк).")
#                 mlflow.log_metric("Recall_40", 0.0)

#         except Exception as e:
#             print(f"ОШИБКА во время запуска ALS с параметрами {current_als_params}: {e}")
#             import traceback
#             traceback.print_exc()
#             mlflow.log_param("error_message", str(e))
#             mlflow.set_tag("run_status", "FAILED") # Используем run_status

# print("\n--- Перебор параметров для ALS завершен ---")

# Перебор парамнтров BPR 

In [ ]:
# from implicit.bpr import BayesianPersonalizedRanking

# def get_bpr_pred(users, nodes, user_to_pred, factors=60, iterations=10, learning_rate=0.01, regularization=0.1, random_state=None):
#     user_ids = users.unique().to_list()
#     item_ids = nodes.unique().to_list()
        
#     user_id_to_index = {user_id: idx for idx, user_id in enumerate(user_ids)}
#     item_id_to_index = {item_id: idx for idx, item_id in enumerate(item_ids)}
#     index_to_item_id = {v:k for k,v in item_id_to_index.items()}
    
#     rows = users.replace_strict(user_id_to_index).to_list()
#     cols = nodes.replace_strict(item_id_to_index).to_list()
#     values = [1] * len(users)
    
#     sparse_matrix = csr_matrix((values, (rows, cols)), shape=(len(user_ids), len(item_ids)))
    
#     model = BayesianPersonalizedRanking(
#         factors=factors,
#         iterations=iterations,
#         learning_rate=learning_rate,
#         regularization=regularization,
#         random_state=random_state
#     )
#     model.fit(sparse_matrix)
    
#     user4pred_indices = np.array([user_id_to_index[i] for i in user_to_pred if i in user_id_to_index]) # Убедимся, что юзеры есть в словаре
#     valid_user_to_pred = [i for i in user_to_pred if i in user_id_to_index]

#     if len(user4pred_indices) == 0:
#          return pl.DataFrame({'node': [], 'cookie': [], 'scores': []})


#     recommendations, scores = model.recommend(
#         user4pred_indices, 
#         sparse_matrix[user4pred_indices], 
#         N=40, 
#         filter_already_liked_items=True
#     )
    
#     df_pred = pl.DataFrame({
#         'node': [[index_to_item_id.get(i) for i in rec_list if index_to_item_id.get(i) is not None] for rec_list in recommendations.tolist()],
#         'cookie': valid_user_to_pred,
#         'scores': scores.tolist()
#     })
#     df_pred = df_pred.explode(['node', 'scores'])
#     return df_pred, model

In [ ]:
# import mlflow
# import itertools
# import time

# param_grid_bpr = {
#     'factors': [150, 200],
#     'iterations': [10, 20],
#     'learning_rate': [0.005, 0.05],
#     'regularization': [0.001, 0.1],
#     'random_state': [42]
# }
# param_names = list(param_grid_bpr.keys())
# param_value_lists = list(param_grid_bpr.values())

# all_param_combinations = list(itertools.product(*param_value_lists))

# print(f"Запланировано запусков для BPR: {len(all_param_combinations)}")

# for i, param_values_tuple in enumerate(all_param_combinations):
#     current_bpr_params = dict(zip(param_names, param_values_tuple))

#     run_name = f"BPR_factors_{current_bpr_params['factors']}" \
#                f"_iter_{current_bpr_params['iterations']}" \
#                f"_lr_{current_bpr_params['learning_rate']}" \
#                f"_reg_{current_bpr_params['regularization']}"

#     print(f"\n--- Запуск {i+1}/{len(all_param_combinations)}: {run_name} ---")
#     print(f"Параметры: {current_bpr_params}")

#     with mlflow.start_run(run_name=run_name):
#         mlflow.log_params(current_bpr_params)

#         try:
#             start_time = time.time()
#             df_pred_bpr, bpr_model_instance = get_bpr_pred(
#                 users, nodes, eval_users, **current_bpr_params
#             )
#             training_time = time.time() - start_time
#             mlflow.log_metric("training_time_sec", training_time)

#             if df_pred_bpr.height > 0:
#                 r_bpr = recall_at(df_eval, df_pred_bpr, k=40)
#                 mlflow.log_metric("Recall_40", r_bpr)
#                 print(f"BPR Recall@40: {r_bpr:.4f} (Обучение: {training_time:.2f} сек)")

#             else:
#                 print("BPR эксперимент: для этого набора параметров предсказания не сгенерированы.")
#                 mlflow.log_metric("Recall_40", 0.0)
#                 mlflow.log_metric("training_time_sec", training_time)

#         except Exception as e:
#             print(f"ОШИБКА во время запуска BPR с параметрами {current_bpr_params}: {e}")
#             mlflow.log_param("error", str(e))

# print("\n--- Перебор параметров для BPR завершен ---")